In [9]:
# Install Tools: use Python with libraries designed for networking
# Scapy (powerful for packet sniffing & crafting)

!pip install scapy          

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Capture Packets
from scapy.all import sniff

# Capture 10 packets
packets = sniff(count=10)
packets.summary()


Ether / IP / TCP 20.42.73.27:https > 192.168.0.2:60069 SA
Ether / IP / TCP 192.168.0.2:60069 > 20.42.73.27:https A
Ether / IP / TCP 192.168.0.2:60069 > 20.42.73.27:https PA / Raw
Ether / IP / TCP 20.189.173.5:https > 192.168.0.2:53207 A / Padding
Ether / IP / TCP 20.189.173.5:https > 192.168.0.2:53207 A / Padding
Ether / IP / TCP 20.189.173.5:https > 192.168.0.2:53207 PA / Raw
Ether / IP / TCP 192.168.0.2:53207 > 20.189.173.5:https PA / Raw
Ether / IP / TCP 20.42.73.27:https > 192.168.0.2:60069 PA / Raw
Ether / IP / TCP 192.168.0.2:60069 > 20.42.73.27:https PA / Raw
Ether / IPv6 / TCP 2620:1ec:33::11:https > 2a06:5900:8101:6a00:e107:5328:aebd:9d84:60006 RA


In [11]:
#  Analyze Packets (overview + detailed)
from scapy.all import sniff

def analyze_packet(packet):
    print(packet.summary())   # quick overview
    packet.show()             # detailed breakdown

sniff(count=10, prn=analyze_packet)


Ether / IP / TCP 52.182.143.208:https > 192.168.0.2:60072 SA
###[ Ethernet ]###
  dst       = 8c:1d:96:d7:0b:0e
  src       = c8:96:5a:18:27:f1
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 25596
     flags     = DF
     frag      = 0
     ttl       = 114
     proto     = tcp
     chksum    = 0x1f97
     src       = 52.182.143.208
     dst       = 192.168.0.2
     \options   \
###[ TCP ]###
        sport     = https
        dport     = 60072
        seq       = 1716151246
        ack       = 3323522053
        dataofs   = 8
        reserved  = 0
        flags     = SA
        window    = 65535
        chksum    = 0x8148
        urgptr    = 0
        options   = [('MSS', 1440), ('NOP', None), ('WScale', 8), ('NOP', None), ('NOP', None), ('SAckOK', b'')]

Ether / IP / TCP 52.182.143.208:https > 192.168.0.2:60071 SA
###[ Ethernet ]###
  dst       = 8c:1d:96:d7:0b:0e
  src       = c8:96:5a:18:27:f1
  type    

<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>

In [12]:
# Extract Useful Info (IP layer)
def analyze_packet(packet):
    if packet.haslayer("IP"):
        ip_layer = packet["IP"]
        print(f"Source: {ip_layer.src} → Destination: {ip_layer.dst}")
        print(f"Protocol: {ip_layer.proto}")  # 6 = TCP, 17 = UDP, etc.

sniff(count=5, prn=analyze_packet)


Source: 170.72.238.46 → Destination: 192.168.0.2
Protocol: 6
Source: 192.168.0.2 → Destination: 170.72.238.46
Protocol: 6
Source: 170.72.238.46 → Destination: 192.168.0.2
Protocol: 6


<Sniffed: TCP:3 UDP:2 ICMP:0 Other:0>

In [14]:
# Add Protocol Names & Payload
def analyze_packet(packet):
    if packet.haslayer("IP"):
        ip_layer = packet["IP"]
        print(f"IP {ip_layer.src} → {ip_layer.dst}")
        
        if packet.haslayer("TCP"):
            print("Protocol: TCP")
            print(f"Source Port: {packet['TCP'].sport}, Dest Port: {packet['TCP'].dport}")
        
        elif packet.haslayer("UDP"):
            print("Protocol: UDP")
            print(f"Source Port: {packet['UDP'].sport}, Dest Port: {packet['UDP'].dport}")
        
        # Show payload (if not empty)
        raw_data = bytes(packet["Raw"].load) if packet.haslayer("Raw") else None
        if raw_data:
            print(f"Payload: {raw_data[:50]}...")  # show first 50 bytes

sniff(count=5, prn=analyze_packet)


IP 209.85.203.157 → 192.168.0.2
Protocol: TCP
Source Port: 443, Dest Port: 65258
IP 209.85.203.157 → 192.168.0.2
Protocol: TCP
Source Port: 443, Dest Port: 52777
IP 209.85.203.157 → 192.168.0.2
Protocol: TCP
Source Port: 443, Dest Port: 54031
IP 192.168.0.2 → 209.85.203.157
Protocol: TCP
Source Port: 52777, Dest Port: 443
IP 192.168.0.2 → 52.182.143.208
Protocol: TCP
Source Port: 59747, Dest Port: 443
Payload: b'\x17\x03\x03\x07\xe1\xab"\xea\x18\x1e\xf6\')\x1c\x9c\x10\x9d\xe6(\xfb\x92\x11\x17\xf1\xf3U\xaf\xc9X\xee\x88\x07\xf2,\xe7\xc9e3\xdb\xa8\xa4}\xc2\x8c\xf6\x94\x06\x979\xb3'...


<Sniffed: TCP:5 UDP:0 ICMP:0 Other:0>

In [15]:
# Filter Traffic (optional)
# Example: capture only HTTP packets (port 80)
sniff(filter="tcp port 80", prn=analyze_packet)


IP 192.168.0.2 → 23.216.155.18
Protocol: TCP
Source Port: 53234, Dest Port: 80
IP 23.216.155.18 → 192.168.0.2
Protocol: TCP
Source Port: 80, Dest Port: 53234
IP 192.168.0.2 → 23.216.155.18
Protocol: TCP
Source Port: 53234, Dest Port: 80
IP 192.168.0.2 → 23.216.155.18
Protocol: TCP
Source Port: 53234, Dest Port: 80
Payload: b'GET /connecttest.txt HTTP/1.1\r\nConnection: Close\r\n'...
IP 23.216.155.18 → 192.168.0.2
Protocol: TCP
Source Port: 80, Dest Port: 53234
IP 192.168.0.2 → 23.216.155.18
Protocol: TCP
Source Port: 53234, Dest Port: 80
IP 23.216.155.18 → 192.168.0.2
Protocol: TCP
Source Port: 80, Dest Port: 53234
IP 23.216.155.18 → 192.168.0.2
Protocol: TCP
Source Port: 80, Dest Port: 53234
Payload: b'HTTP/1.1 200 OK\r\nContent-Length: 22\r\nDate: Fri, 29'...
IP 192.168.0.2 → 23.216.155.18
Protocol: TCP
Source Port: 53234, Dest Port: 80
IP 23.216.155.18 → 192.168.0.2
Protocol: TCP
Source Port: 80, Dest Port: 53234
IP 192.168.0.2 → 23.216.155.18
Protocol: TCP
Source Port: 53234, Dest 

<Sniffed: TCP:82 UDP:0 ICMP:0 Other:0>

Test & Learn Notes

- Run the program and browse a website → see packets appear.  
- Try pinging another computer → notice ICMP packets.  
- Watch how many small TCP/UDP packets carry even simple messages.